#### Titanic: Machine Learning from Disaster
This Kaggle competition is about predicting the survivors of Titanic from their socio-economic or demographic data. The team will analyze and understand the data prior to developing predictive models later submit results on Kaggle and compete with other Kagglers. Code will be shared on GitHub repositories also. <br/>

Access to competition via <https://www.kaggle.com/c/titanic/overview>

### Data Import
Kaggle API is used to download and access to the data. Same API will be used later to commit notebook files on GitHub and Kaggle also.<br />
<b/>See Kaggle API Documentay at https://github.com/Kaggle/kaggle-api

In [1]:
!kaggle competitions download -c titanic

titanic.zip: Skipping, found more recently modified local copy (use --force to force download)


  * Files downloaded as .zip files, code below will extract them in the directory. <br />
  train.csv, test.csv and sample submission file gender_submission.csv files are now ready to use.

In [2]:
import os
from zipfile import ZipFile
with ZipFile('titanic.zip','r') as ZipObj:
    ZipObj.extractall()
    
files = os.listdir()
print(files)

['.git', '.gitignore', '.ipynb_checkpoints', 'gender_submission.csv', 'Kaggle - Titanic Surviver Prediction.ipynb', 'README.md', 'test.csv', 'titanic.zip', 'train.csv']


#### Library Imports 

In [3]:
import pandas as pd
import plotly as plt
import numpy as np
import scipy as sci
import plotly

## 1. Checking Data

### Data Dictionary

<strong>Variable /	Definition	  /                                Key</strong><br/>
survival /	Survival	/                                  0 = No, 1 = Yes<br/>
pclass	 /   Ticket class /	                              1 = 1st, 2 = 2nd, 3 = 3rd<br/>
sex	      /  Sex	             /                             -<br/>
Age	    /    Age in years	    /                          -<br/>
sibsp	 /   # of siblings - spouses aboard the Titanic	/  -<br/>
parch	 /   # of parents - children aboard the Titanic	 / -<br/>
ticket	 /   Ticket number	            /                  - <br/>
fare	 /   Passenger fare	            /                  -<br/>
cabin	 /   Cabin number	           /                   -<br/>
embarked /	Port of Embarkation	       /                   C = Cherbourg, Q = Queenstown, S = Southampton<br/>

#### Variable Notes
<strong>pclass:</strong> A proxy for socio-economic status (SES)<br/>
1st = Upper<br/>
2nd = Middle<br/>
3rd = Lower<br/>

<strong>age</strong>: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

<strong>sibsp</strong>: The dataset defines family relations in this way...<br/>
Sibling = brother, sister, stepbrother, stepsister<br/>
Spouse = husband, wife (mistresses and fiancés were ignored)<br/>

<strong>parch</strong>: The dataset defines family relations in this way...<br/>
Parent = mother, father<br/>
Child = daughter, son, stepdaughter, stepson<br/>
Some children travelled only with a nanny, therefore parch=0 for them.<br/>

In [4]:
df = pd.read_csv('train.csv')
df.head()

#In describe summary this name comes as a TOP name. This is just to see its rows -only one row interestingly-
#cabin_G6 = df[df.Name.str.contains('Johnson, Mrs. Oscar W')]
#cabin_G6.head(50)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df.describe(include='all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Attalah, Miss. Malake",male,NaN,NaN,NaN,CA. 2343,NaN,G6,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


Missing values are present in the Age and Cabin columns. Since both variables can be important predictors, they will be dealed with.<br/>
for the Age column, missing values can be created via analyzing SibSp and Parch columns. Same approach can be useful for the Cabin column since it is expected for the families stay in the same cabin.